In [105]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


def transform(df, mac):
    df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
    packets_per_min = df.set_index('Epoch Time').groupby(pd.Grouper(freq='15s'))['Length'].mean()
    packets_per_min = packets_per_min.fillna(0)

    time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq='15s'))['Delta Time'].mean()
    time_deltas = time_deltas.fillna(time_deltas.mean())
    df = pd.merge(packets_per_min, time_deltas, left_index=True, right_index=True)

    df['MAC Source'] = mac
    return df


data = r'C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-1h_1.csv'
df = pd.read_csv(data)
df = df[df['MAC Source']=='00:0c:29:9d:9e:9e']
name = df['MAC Source'].unique()[0]

df = transform(df, name)

x = df['Delta Time'].values



In [106]:
# define base model
def baseline_model():
	# # create model
	# model = Sequential()
	# model.add(Dense(12, input_dim=1, activation='relu'))
	# model.add(Dense(8, activation='relu'))
	# model.add(Dense(1, activation='sigmoid'))
	# # Compile model
	# model.compile(loss='categorical_crossentropy')
	model = DecisionTreeClassifier()
	return model

In [107]:
from sklearn import preprocessing
# estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
# kfold = KFold(n_splits=10)
# results = cross_val_score(estimator, x, x, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

model = baseline_model()
# model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

x_str = [str(i) for i in x]

model.fit(np.array(x).reshape(-1,1), x_str)


# y_pred = model.predict(np.array(x).reshape(-1,1))
# df = pd.DataFrame()
# df['x'] = x
# df['y'] = y_pred
# y_pred = [float(i) for i in y_pred]
# df['Diff'] = abs(np.array(x)-np.array(y_pred))
# df.to_csv('test.csv', index=False)

# acc = np.sum(abs(np.array(x)-np.array(y_pred)))





DecisionTreeClassifier()

In [111]:
data = r'C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-6h_1.csv'
df = pd.read_csv(data)
df = df[df['MAC Source']=='00:0c:29:9d:9e:9e']
name = df['MAC Source'].unique()[0]

df = transform(df, name)

x = df['Delta Time'].values
y_pred = model.predict(np.array(x).reshape(-1,1))
df = pd.DataFrame()
df['x'] = x
df['y'] = y_pred
y_pred = [float(i) for i in y_pred]
df['Diff'] = abs(np.array(x)-np.array(y_pred))
df.to_csv('test.csv', index=False)

acc = np.sum(abs(np.array(x)-np.array(y_pred)))
acc


0.17756858471713854